In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import nest_asyncio
from rich  import print as pprint
nest_asyncio.apply()

In [3]:
from pydantic_ai import Agent

demo_agent = Agent(
    model="groq:llama-3.2-3b-preview",
    system_prompt= "You are an excellent cook."
)

result = demo_agent.run_sync("Suggest me 3 recipe for today's breakfast")

In [4]:
pprint(result.data)

I'd be happy to suggest three delicious breakfast recipes for you. Here they are:

**Recipe 1: Avocado and Bacon Omelette**

Ingredients:
- 2 eggs
- 1/4 avocado, diced
- 4 slices of cooked bacon, crumbled
- Salt and pepper to taste
- Optional: cheese and chives for garnish

Instructions:
1. Whisk eggs together and set aside.
2. In a pan, cook bacon over medium heat until crispy.
3. Add diced avocado and cook for 1 minute.
4. Pour whisked eggs over the bacon and avocado mixture.
5. Cook until eggs are set, then fold the omelette in half.
6. Season with salt and pepper, and garnish with cheese and chives.

**Recipe 2: Cinnamon French Toast with Fresh Berries**

Ingredients:
- 4 slices of bread
- 1/4 cup granulated sugar
- 2 eggs
- 1/2 cup milk
- 1 tsp ground cinnamon
- 4-6 fresh berries (strawberries, blueberries, raspberries)
- Maple syrup (optional)

Instructions:
1. In a shallow dish, whisk together eggs, milk, and cinnamon.
2. Heat a non-stick pan or griddle over medium heat.
3. Dip each bread slice into the egg mixture, coating both sides.
4. Cook until golden brown on both sides.
5. Serve with fresh berries and drizzle with maple syrup, if desired.

**Recipe 3: Spinach, Mushroom, and Feta Quiche**

Ingredients:
- 1 9-inch pie crust
- 3 cups fresh spinach, chopped
- 1 cup sliced mushrooms
- 1/2 cup feta cheese, crumbled
- 6 eggs
- 1/2 cup grated cheddar cheese

Instructions:
1. Preheat oven to 375°F (190°C).
2. Roll out the pie crust and place in a pie dish.
3. In a pan, sauté mushrooms and spinach until tender.
4. Whisk eggs together and season with salt and pepper.
5. Add crumbled feta cheese, cooked mushroom and spinach mixture, and grated cheddar cheese to the eggs.
6. Pour the egg mixture into the pie crust and bake for 35-40 minutes or until set.

Which one of these recipes sounds appealing to you?

In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import html2text
import re

def clean_text(text):
    """Clean extracted text by removing extra whitespace and empty lines"""
    text = re.sub(r'\n\s*\n', '\n\n', text.strip())
    return text

def scrape_website(url, selector=None):
    """
    Scrapes data from a website and converts HTML to Markdown.
    
    Parameters:
    url (str): The URL of the website to scrape
    selector (str, optional): CSS selector to target specific elements
    
    Returns:
    str: Markdown formatted text
    """
    try:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        }
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Remove unwanted elements
        for element in soup.select('script, style, nav, footer, header'):
            element.decompose()
        
        # Convert to Markdown
        h = html2text.HTML2Text()
        h.ignore_links = False
        h.ignore_images = False
        h.body_width = 0  # No wrapping
        
        if selector:
            elements = soup.select(selector)
            content = '\n\n'.join(h.handle(str(element)) for element in elements)
        else:
            content = h.handle(str(soup.body))
        
        # Clean and save content
        content = clean_text(content)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        with open(f'scraped_content_{timestamp}.md', 'w', encoding='utf-8') as file:
            file.write(content)   
        return content
        
    except Exception as e:
        print(f"Error: {e}")
        return ""

website_url = "https://shivance.github.io/pages/authoring.html"
content = scrape_website(website_url)

What is given?

url - web address to the job posting

What is agent supposed to do?

1. Fetch the job data form the website url
2. Generate a cold email for the given job

In [6]:
from dataclasses import dataclass
from typing import List
from pydantic import BaseModel , Field

# Tool input
@dataclass
class JobInformationFetchDeps():
    job_post_url : str
    
# Tool Output Agent Input
@dataclass
class JobDescriptionAgentDependencies:
    job_posting_information : str
    
class JobDescriptionAgentResult(BaseModel):
    role : str = Field(
        description = "The job title or role position being described (e.g., 'Senior Software Engineer', 'Product Manager')"
    )
    company_name :str = Field(
        description = "Name of the company which is providing job"
    )
    experience : str = Field(
        description = "Required years and type of experience for the position (e.g., '5+ years of software development')"
    )
    skills : List[str]  = Field(
        description = "List of specific technical skills, tools, or competencies required for the role (e.g., ['Python', 'AWS', 'Machine Learning'])"
    )
    description : str = Field(
        description = "Detailed overview of the job responsibilities, requirements, and expectations"
    )

In [7]:
job_description_parser_agent = Agent(
    model = 'groq:llama3-70b-8192',
    deps_type=JobInformationFetchDeps,
    result_type=JobDescriptionAgentResult,
    system_prompt="""You are a specialized HR assistant focused on analyzing and structuring job descriptions. Your primary responsibilities are:
1. Use the get_job_details tool to retrieve job posting information
2. Extract and categorize key components including:
   - Core role/position title
   - Required experience level
   - Essential skills and qualifications
   - Detailed role description and responsibilities
Format all outputs according to the JobDescription schema. Be precise and consistent in your categorization. When analyzing skills:
- Focus on specific technical and professional competencies
- Separate distinct skills into individual items
- Standardize skill names (e.g., "Python" not "python programming")
If job details are ambiguous or incomplete, make reasonable inferences based on industry standards while maintaining accuracy.""",
)

In [8]:
from pydantic_ai import RunContext
@job_description_parser_agent.tool
def get_job_details(
    ctx: RunContext[JobInformationFetchDeps],
) -> JobDescriptionAgentDependencies:
    """
    Retrieves and extracts job posting information
    """
    job_post_url = ctx.deps.job_post_url
    job_posting_information = scrape_website(url=job_post_url)
    return JobDescriptionAgentDependencies(
        job_posting_information=job_posting_information
    )

In [9]:
# Verify the URL and scrape the website content
job_post_url = "https://boards.greenhouse.io/anthropic/jobs/4042814008"
scraped_content = scrape_website(job_post_url)

# Check if the scraping function returned the expected result
if scraped_content:
    job_description_parser_agent_result = job_description_parser_agent.run_sync(
        user_prompt="Please extract job description for the provided URL",
        deps=JobInformationFetchDeps(job_post_url=job_post_url)
    )
    if job_description_parser_agent_result:
        pprint(job_description_parser_agent_result.data)
    else:
        print("Failed to extract job description.")
else:
    print("Failed to scrape the job posting information.")

JobDescriptionAgentResult(
    role='Software Engineer, Claude.AI',
    company_name='Anthropic',
    experience='5+ years of experience as a software engineer, preferably building APIs and/or interfaces',
    skills=[
        'React',
        'Service-oriented architectures',
        'Rest APIs',
        'Containerization',
        'DevOps',
        'QA and testing automation',
        'Privacy and security best practices'
    ],
    description='Retrieves and extracts job posting information. Develops powerful solutions that leverage the 
power of language models to streamline tasks and boost productivity.'
)

In [15]:
class ColdEmailWriterAgentInput(BaseModel):
    job_description: JobDescriptionAgentResult = Field(
        description="Parsed job posting details including role, company, required experience, skills, and full description"
    )
class ColdEmailWriterAgentResponse(BaseModel):
    subject: str = Field(
        description="Email subject line that captures attention and highlights key value proposition (e.g., 'Experienced Python/ML Engineers Available for Anthropic's AI Initiative')"
    )
    body: str = Field(
        description="Professional email body that matches job requirements with portfolio expertise, includes introduction, value proposition, relevant project examples, and call-to-action"
    )
    
cold_email_writer_agent = Agent(
    model='groq:llama3-70b-8192',
    deps_type=ColdEmailWriterAgentInput,
    result_type=ColdEmailWriterAgentResponse,
    system_prompt="""
You are Ria, a tech recruitment specialist at Turing, reaching out to hiring managers about your firm's pre-vetted engineering talent pool. Using the provided job description:

1. Analyze role requirements and highlight relevant Turing portfolio projects
2. Create concise, compelling subject lines highlighting available talent
3. Write brief, impactful email body (3-4 paragraphs max) that:
   - Opens with specific reference to company's hiring needs
   - Showcases relevant Turing portfolio projects matching required tech stack
   - Emphasizes that Turing has pre-vetted engineers ready to interview
   - Includes clear call-to-action to discuss available candidates

Keep tone professional yet conversational. Focus on Turing's talent pool and proven project experience.
"""
)

In [16]:
# Ensure job_description_parser_agent_result.data is correctly formatted
job_description_data = job_description_parser_agent_result.data

# Run the cold email writer agent
result = cold_email_writer_agent.run_sync(
    "Please write a cold email",
    deps=ColdEmailWriterAgentInput(
        job_description=job_description_data
    )
)

# Print the result
pprint(result.data)

ColdEmailWriterAgentResponse(
    subject="Experienced Python/ML Engineers Available for Anthropic's AI Initiative",
    body="Hi [Hiring Manager's Name],\n\nI came across Anthropic's AI initiative and noticed that you're looking 
for experienced engineers to join your team. At Turing, we've worked with various companies to help them find the 
best talent for their AI/ML projects.\n\nOur portfolio includes projects like [Portfolio Project 1], which 
demonstrates expertise in Python and machine learning. We also have engineers who have worked on [Portfolio Project
2], showcasing their skills in developing AI models.\n\nWe have pre-vetted engineers ready to interview, and I'd 
love to discuss how they can contribute to your project. Would you be open to a quick call to explore this 
further?\n\nBest,\nRia"
)

In [17]:
pprint(result.data.body)

Hi [Hiring Manager's Name],

I came across Anthropic's AI initiative and noticed that you're looking for experienced engineers to join your 
team. At Turing, we've worked with various companies to help them find the best talent for their AI/ML projects.

Our portfolio includes projects like [Portfolio Project 1], which demonstrates expertise in Python and machine 
learning. We also have engineers who have worked on [Portfolio Project 2], showcasing their skills in developing AI
models.

We have pre-vetted engineers ready to interview, and I'd love to discuss how they can contribute to your project. 
Would you be open to a quick call to explore this further?

Best,
Ria